# Домашнее задание к занятию 2.1: Деревья решений. Классификация

## Домашнее задание

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz

**Lvl 1:**

* взять подготовленные раннее данные из задачи **Titanic**, обучиться на них с помощью дерева решений и кросс-валидации и сделать сабмит
* кросс-валидацию желательно сделать сразу по нескольким фичам ( параметр *grid* в *GridSearchCV* )
* определить самые важные фичи
* вывести дерево решений (можете попробовать установить pydot и webgraphviz для отрисовки деревьев внутри ноутбука)

**Результат:** скрины нового сабмита на Kaggle и построенного дерева

In [2]:
X_train = pd.read_csv('./titanic/train.csv')
X_test = pd.read_csv('./titanic/test.csv')

In [3]:
X_testy = pd.read_csv('./titanic/test.csv')

In [4]:
y_train=X_train['Survived']
X_train.drop(['Survived'], axis=1,inplace=True)

In [5]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
X_train['isMale'] = X_train.Sex.replace({'male': 1, 'female':0})

In [7]:
X_train['hasCabin']=(~X_train['Cabin'].isnull()).astype(int)

In [8]:
imputer = SimpleImputer(strategy='mean')
X_train[['Age']]=imputer.fit_transform(X_train[['Age']])
X_test[['Age']]=imputer.transform(X_test[['Age']])

In [9]:
X_train=pd.get_dummies(X_train, columns=['Pclass', 'Embarked'])

In [10]:
X_train.drop(['PassengerId','Name','Sex','Ticket','Cabin'],axis=1,inplace=True)

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
isMale        891 non-null int64
hasCabin      891 non-null int64
Pclass_1      891 non-null uint8
Pclass_2      891 non-null uint8
Pclass_3      891 non-null uint8
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: float64(2), int64(4), uint8(6)
memory usage: 47.1 KB


In [12]:
tree = DecisionTreeClassifier()
tree_params={'max_depth':range(1,10),'max_features':range(2,12)}
tree_grid=GridSearchCV(tree,tree_params, cv=5)
tree_grid.fit(X_train,y_train)
tree_grid.best_params_,tree_grid.best_score_

//anaconda3/envs/tuo/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'max_depth': 8, 'max_features': 9}, 0.8282828282828283)

In [13]:
results = pd.DataFrame(tree_grid.cv_results_)
results.sort_values('mean_test_score',ascending=False).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
77,0.002622,0.000042,0.001069,0.000051,8,9,"{'max_depth': 8, 'max_features': 9}",0.843575,0.782123,0.853933,0.814607,0.847458,0.828283,0.026775,1
78,0.002687,0.000047,0.001096,0.000078,8,10,"{'max_depth': 8, 'max_features': 10}",0.798883,0.810056,0.853933,0.775281,0.853107,0.818182,0.030878,2
76,0.002572,0.000128,0.001028,0.000028,8,8,"{'max_depth': 8, 'max_features': 8}",0.798883,0.776536,0.865169,0.803371,0.841808,0.817059,0.031902,3
58,0.002603,0.000106,0.001033,0.000052,6,10,"{'max_depth': 6, 'max_features': 10}",0.821229,0.815642,0.808989,0.780899,0.853107,0.815937,0.023130,4
47,0.003151,0.000602,0.001460,0.000268,5,9,"{'max_depth': 5, 'max_features': 9}",0.798883,0.832402,0.814607,0.786517,0.836158,0.813692,0.019036,5


In [14]:
p=[tree_grid.best_params_['max_depth'],tree_grid.best_params_['max_features']]
clf=DecisionTreeClassifier(max_depth=p[0],max_features=p[1])
clf.fit(X_train,y_train)
export_graphviz(clf, feature_names=X_train.columns, 
out_file='tree.dot', filled=True)
!dot -Tpng 'tree.dot' -o 'tree.png'
clf.feature_importances_

array([0.18972221, 0.03699034, 0.01319519, 0.14450326, 0.41270452,
       0.03050919, 0.00332023, 0.00353392, 0.15472153, 0.00374312,
       0.        , 0.00705648])

![title](tree.png)

In [15]:
X_test['isMale'] = X_test.Sex.replace({'male': 1, 'female':0})
X_test['hasCabin']=(~X_test['Cabin'].isnull()).astype(int)
X_test=pd.get_dummies(X_test, columns=['Pclass', 'Embarked'])
X_test.drop(['PassengerId','Name','Sex','Ticket','Cabin'],axis=1,inplace=True)
X_test['Fare'].fillna(X_test['Fare'].mean(),inplace=True)

In [16]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
Age           418 non-null float64
SibSp         418 non-null int64
Parch         418 non-null int64
Fare          418 non-null float64
isMale        418 non-null int64
hasCabin      418 non-null int64
Pclass_1      418 non-null uint8
Pclass_2      418 non-null uint8
Pclass_3      418 non-null uint8
Embarked_C    418 non-null uint8
Embarked_Q    418 non-null uint8
Embarked_S    418 non-null uint8
dtypes: float64(2), int64(4), uint8(6)
memory usage: 22.2 KB


In [23]:
predictions=pd.DataFrame(clf.predict(X_test),columns=['Survived'])
predictions['PassengerId']=X_testy[['PassengerId']]
predictions=predictions[['PassengerId','Survived']]
predictions.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,1


In [26]:
predictions.to_csv(r'./titanic/predictions.csv', index=False)

**Lvl 2:** (опционально)

* С помощью функций sklearn.metrics.auc, precision, recall составить функцию для расчёта ROC-AUC, ROC-PRC
* Придумать себе интересную задачу на основе данных из интернета =) Спарсить ещё какой-нибудь сайт (не Ведомости) и решить задачу классификации. Делать свои проекты - круто. Если будут - кидайте мне =) [@NikitaKuznetsov](http://t.me/NikitaKuznesov)